
# 🚇 Sürücüsüz Metro Simülasyonu (Rota Optimizasyonu)

Bu notebook, bir metro ağı üzerinde **en az aktarmalı** ve **en hızlı rotayı** bulmak için iki farklı algoritma uygular:
- **BFS (Breadth-First Search)**: En az aktarmalı rotayı bulur.
- **A* Algoritması**: En hızlı rotayı bulur.

### 📌 Kullanılan Kütüphaneler:
- `collections` (BFS için kuyruk yapısı)
- `heapq` (A* algoritması için öncelik kuyruğu)
- `networkx, matplotlib` (Graf görselleştirmesi)



In [ ]:

import heapq
import networkx as nx
import matplotlib.pyplot as plt
from collections import deque

class MetroAgi:
    def __init__(self):
        self.graph = {}
        self.durations = {}

    def istasyon_ekle(self, istasyon):
        if istasyon not in self.graph:
            self.graph[istasyon] = []

    def baglanti_ekle(self, istasyon1, istasyon2, sure):
        self.graph[istasyon1].append(istasyon2)
        self.graph[istasyon2].append(istasyon1)
        self.durations[(istasyon1, istasyon2)] = sure
        self.durations[(istasyon2, istasyon1)] = sure

    def grafiği_görselleştir(self):
        G = nx.Graph()
        for istasyon, komsular in self.graph.items():
            for komsu in komsular:
                G.add_edge(istasyon, komsu, weight=self.durations[(istasyon, komsu)])
        
        pos = nx.spring_layout(G)
        labels = nx.get_edge_attributes(G, 'weight')
        nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=2000, font_size=10)
        nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
        plt.show()


In [ ]:

def en_az_aktarma_bul(metro, baslangic, hedef):
    kuyruk = deque([(baslangic, [baslangic])])
    ziyaret_edilen = set()

    while kuyruk:
        istasyon, yol = kuyruk.popleft()
        if istasyon == hedef:
            return yol

        for komsu in metro.graph[istasyon]:
            if komsu not in ziyaret_edilen:
                ziyaret_edilen.add(komsu)
                kuyruk.append((komsu, yol + [komsu]))

    return None  # Yol bulunamazsa


In [ ]:

def en_hizli_rota_bul(metro, baslangic, hedef):
    oncelik_kuyrugu = [(0, baslangic, [baslangic])]
    ziyaret_edilen = {}

    while oncelik_kuyrugu:
        mevcut_sure, istasyon, yol = heapq.heappop(oncelik_kuyrugu)

        if istasyon == hedef:
            return yol

        for komsu in metro.graph[istasyon]:
            yeni_sure = mevcut_sure + metro.durations[(istasyon, komsu)]
            if komsu not in ziyaret_edilen or yeni_sure < ziyaret_edilen[komsu]:
                ziyaret_edilen[komsu] = yeni_sure
                heapq.heappush(oncelik_kuyrugu, (yeni_sure, komsu, yol + [komsu]))

    return None


In [ ]:

# Metro ağı oluşturma
metro = MetroAgi()
istasyonlar = ["A", "B", "C", "D", "E", "F"]
for istasyon in istasyonlar:
    metro.istasyon_ekle(istasyon)

# Bağlantılar ekleniyor
metro.baglanti_ekle("A", "B", 4)
metro.baglanti_ekle("A", "C", 2)
metro.baglanti_ekle("B", "D", 5)
metro.baglanti_ekle("C", "D", 1)
metro.baglanti_ekle("D", "E", 3)
metro.baglanti_ekle("E", "F", 2)

# Metro ağını görselleştir
metro.grafiği_görselleştir()

# BFS (En az aktarmalı rota)
print("En az aktarmalı rota (BFS):", en_az_aktarma_bul(metro, "A", "F"))

# A* (En hızlı rota)
print("En hızlı rota (A*):", en_hizli_rota_bul(metro, "A", "F"))


In [ ]:

import time

def a_star_canli_simulasyon(metro, baslangic, hedef):
    oncelik_kuyrugu = [(0, baslangic, [baslangic])]
    ziyaret_edilen = {}
    fig, ax = plt.subplots(figsize=(8, 6))
    
    while oncelik_kuyrugu:
        mevcut_sure, istasyon, yol = heapq.heappop(oncelik_kuyrugu)

        # Grafiği her adımda güncelle
        G = nx.Graph()
        for ist, komsular in metro.graph.items():
            for komsu in komsular:
                G.add_edge(ist, komsu, weight=metro.durations[(ist, komsu)])
        
        pos = nx.spring_layout(G)
        labels = nx.get_edge_attributes(G, 'weight')

        ax.clear()
        nx.draw(G, pos, with_labels=True, node_color='lightgray', node_size=2000, font_size=10)
        nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
        
        # Şu ana kadar geçen yolu farklı renkte göster
        path_edges = list(zip(yol, yol[1:]))
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='r', width=2)
        plt.pause(0.5)  # Adım adım ilerleme

        if istasyon == hedef:
            plt.show()
            return yol

        for komsu in metro.graph[istasyon]:
            yeni_sure = mevcut_sure + metro.durations[(istasyon, komsu)]
            if komsu not in ziyaret_edilen or yeni_sure < ziyaret_edilen[komsu]:
                ziyaret_edilen[komsu] = yeni_sure
                heapq.heappush(oncelik_kuyrugu, (yeni_sure, komsu, yol + [komsu]))

    plt.show()
    return None


In [ ]:

def istatistikleri_hesapla(metro, test_senaryolari):
    toplam_sureler = []
    en_cok_gecen_istasyonlar = {}

    for baslangic, hedef in test_senaryolari:
        rota = en_hizli_rota_bul(metro, baslangic, hedef)
        if rota:
            sure = sum(metro.durations[(rota[i], rota[i+1])] for i in range(len(rota)-1))
            toplam_sureler.append(sure)

            for istasyon in rota:
                if istasyon not in en_cok_gecen_istasyonlar:
                    en_cok_gecen_istasyonlar[istasyon] = 0
                en_cok_gecen_istasyonlar[istasyon] += 1

    ortalama_sure = sum(toplam_sureler) / len(toplam_sureler) if toplam_sureler else 0
    en_cok_kullanilan_istasyon = max(en_cok_gecen_istasyonlar, key=en_cok_gecen_istasyonlar.get)

    print(f"📊 Ortalama Seyahat Süresi: {ortalama_sure:.2f} dakika")
    print(f"🚉 En Çok Kullanılan İstasyon: {en_cok_kullanilan_istasyon}")


In [ ]:

# A* Algoritması için Canlı Simülasyon Testi
print("A* Algoritması Canlı Simülasyonu (Adım Adım Görselleştirme)")
a_star_canli_simulasyon(metro, "A", "F")

# İstatistikleri hesapla
test_senaryolari = [("A", "F"), ("B", "E"), ("C", "D")]
istatistikleri_hesapla(metro, test_senaryolari)
